In [1]:
import psycopg2
from configparser import ConfigParser

In [2]:
# connect to database
#try:
#    conn = psycopg2.connect("dbname='w4111' user='ho2271' host='w4111.cisxo09blonu.us-east-1.rds.amazonaws.com' password='d1d6s4ad'")
#except:
#    print ("I am unable to connect to the database")

In [3]:
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

In [6]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        DROP TABLE IF EXISTS website CASCADE;
        CREATE TABLE website( 
        web_id VARCHAR(255),
        PRIMARY KEY (web_id)
        );
        """,
        """ 
        DROP TABLE IF EXISTS movie CASCADE;
        CREATE TABLE movie( 
        name VARCHAR(255), 
        mov_id int,
        language text, 
        runtime int,
        release_date date, 
        revenue bigint, 
        PRIMARY KEY (mov_id) 
        );
        """,
        """
        DROP TABLE IF EXISTS link CASCADE;
        CREATE TABLE link( 
        mov_id int,
        web_id VARCHAR(255),
        PRIMARY KEY (mov_id, web_id),
        FOREIGN KEY (mov_id) REFERENCES movie (mov_id), 
        FOREIGN KEY (web_id) REFERENCES website (web_id) 
        );
        """,
        """
        DROP TABLE IF EXISTS genre CASCADE;
        CREATE TABLE genre( 
        name VARCHAR(255), 
        genre_id int,
        PRIMARY KEY (genre_id)
        );
        """,
        """
        DROP TABLE IF EXISTS user_most_like CASCADE;
        CREATE TABLE user_most_like( 
        name VARCHAR(255),
        user_id int,
        genre_id int,
        gender text,
        birthday date, 
        PRIMARY KEY (user_id),
        FOREIGN KEY (genre_id) REFERENCES genre(genre_id),
        CHECK(now()::date > birthday AND birthday > date '1900-01-01') 
        );
        """,
        """
        DROP TABLE IF EXISTS rate CASCADE;
        CREATE TABLE rate( 
        mov_id int,
        user_id int,
        grade int,
        review text,
        PRIMARY KEY (mov_id, user_id),
        FOREIGN KEY (mov_id) REFERENCES movie (mov_id),
        FOREIGN KEY (user_id) REFERENCES user_most_like(user_id),
        CHECK (5>=grade AND grade >=0) 
        );
        """, 
        """
        DROP TABLE IF EXISTS belong_to CASCADE;
        CREATE TABLE belong_to(
        mov_id int,
        genre_id int,
        PRIMARY KEY (mov_id, genre_id),
        FOREIGN KEY (mov_id) REFERENCES movie (mov_id),
        FOREIGN KEY (genre_id) REFERENCES genre(genre_id)
        );
        """,
        """
        DROP TABLE IF EXISTS director CASCADE;
        CREATE TABLE director(
        name VARCHAR(255),
        director_id int,
        gender int, 
        PRIMARY KEY (director_id),
        CHECK (gender=2 OR gender=1 OR gender=0)
        );
        """,
        """
        DROP TABLE IF EXISTS mov_cast CASCADE;
        CREATE TABLE mov_cast( 
        name VARCHAR(255),
        cast_id int,
        gender text,
        PRIMARY KEY (cast_id)
        );
        """,
        """
        DROP TABLE IF EXISTS direct CASCADE;
        CREATE TABLE direct(
        mov_id int,
        director_id int,
        PRIMARY KEY (mov_id, director_id),
        FOREIGN KEY (mov_id) REFERENCES movie (mov_id),
        FOREIGN KEY (director_id) REFERENCES director (director_id)
        );
        """,
        """
        DROP TABLE IF EXISTS act CASCADE;
        CREATE TABLE act(
        mov_id int,
        role text,
        cast_id int,
        PRIMARY KEY (mov_id, cast_id),
        FOREIGN KEY (mov_id) REFERENCES movie (mov_id),
        FOREIGN KEY (cast_id) REFERENCES mov_cast (cast_id)
        );
        """
    )
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [7]:
create_tables()

In [8]:
def insert_csv(file_path, table_name):
    """ insert csv into tables in the PostgreSQL database"""
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # insert csv files
        f = open(file_path, "r")
        # Truncate the table first
        cur.execute("Truncate {} Cascade;".format(table_name))
        print("Truncated {}".format(table_name))
        # Load table from the file with header
        cur.copy_expert("copy {} from STDIN CSV HEADER QUOTE '\"'".format(table_name), f)
        print("Loaded data into {}".format(table_name))
        # close communication with the PostgreSQL database server
        cur.close()
        f.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [9]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_movie.csv', 'movie')

Truncated movie
Loaded data into movie


In [10]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_genre.csv', 'genre')

Truncated genre
Loaded data into genre


In [11]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_director.csv', 'director')

Truncated director
Loaded data into director


In [12]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_direct.csv', 'direct')

Truncated direct
Loaded data into direct


In [13]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_mov_cast.csv', 'mov_cast')

Truncated mov_cast
Loaded data into mov_cast


In [14]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_act.csv', 'act')

Truncated act
Loaded data into act


In [15]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_belong_to.csv', 'belong_to')

Truncated belong_to
Loaded data into belong_to


In [16]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_user_most_like.csv', 'user_most_like')

Truncated user_most_like
Loaded data into user_most_like


In [17]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_rate.csv', 'rate')

Truncated rate
Loaded data into rate


In [18]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_website.csv', 'website')

Truncated website
Loaded data into website


In [19]:
insert_csv('/home/ho2271/project1/data_preprocessing/table_link.csv', 'link')

Truncated link
Loaded data into link
